In [ ]:
import re
import pandas as pd
import numpy as np
import gdown
import matplotlib.pyplot as plt

In [ ]:
!gdown --id '1FHsZ7ZzsIg5ufBjrNjUi5UfOL2tisBaK' --output data.zip
!gdown --id '1gxVJ4WyLwdg1_xM5TATBDNY9oxaHIrxT' --output area.xlsx
# !wget -O TaipeiSansTCBeta-Regular.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download
!unzip 'data.zip'

Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1FHsZ7ZzsIg5ufBjrNjUi5UfOL2tisBaK 

Downloading...
From: https://drive.google.com/uc?id=1gxVJ4WyLwdg1_xM5TATBDNY9oxaHIrxT
To: /content/area.xlsx
100% 177k/177k [00:00<00:00, 28.6MB/s]
Archive:  data.zip
replace data/1962_jieba.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/1962_jieba.xlsx    
  inflating: data/1961_jieba.xlsx    
  inflating: data/1959_jieba.xlsx    
  inflating: data/1960_jieba.xlsx    
  inflating: data/1958_jieba.xlsx    


In [ ]:
df0 = pd.read_excel('data/'+'1958_jieba.xlsx')
df1 = pd.read_excel('data/'+'1959_jieba.xlsx')
df2 = pd.read_excel('data/'+'1960_jieba.xlsx')
df3 = pd.read_excel('data/'+'1961_jieba.xlsx')
df4 = pd.read_excel('data/'+'1962_jieba.xlsx')
df = pd.concat([df0, df1, df2, df3, df4],axis=0, ignore_index=True)

df = df.dropna(subset=["title",'doc_content','tfidf']) #去除遺漏值
print(f'資料量：　{len(df)}')
df

資料量：　112484


,Unnamed: 0,Unnamed: 0.1,filename,corpus,timeseq_not_before,era,compilation_vol,docclass,doctype,compilation_name,author,title,doc_content,token,pos,tfidf
0,0,0,人民日報_19580101_1_184147,1958,19580101,195801,第1版,NaN,社论,NaN,NaN,乘风破浪,人们的思想常常落后于实际，对于客观形势发展之快估计不足。回顾1957年，国际和国内形势的发展...,人们 的 思想 常常 落后 于 实际 ， 对于 客观 形势 发展 之 快 估计不足 。 回顾...,n uj n d a p n x p n n vn u a v x v m m x n c ...,"整风运动,1957,五年计划,力量,胜利,帝国主义,和平,亚非,运动,发展,革命,共产主义,..."
1,1,1,人民日報_19580101_5_184208,1958,19580101,195801,第5版,NaN,NaN,NaN,NaN,流年不利,方成钟灵,方成 钟灵,nrfg nr,"钟灵,方成"
2,2,2,人民日報_19580101_5_184207,1958,19580101,195801,第5版,NaN,扎记,NaN,所提出的,次等伙伴的悲哀,在回答“美国新闻与世界报道”杂志的记者所提出的，美国能够做些什么来“帮助”法国这一问题的时候...,在 回答 “ 美国 新闻 与 世界 报道 ” 杂志 的 记者 所 提出 的 ， 美国 能够 ...,p v x ns n p n v x n uj n c v uj x ns v v q r ...,"盖伊,美国,北非,美国政府,可怜,阿说,阿在,多费,法国总理,一百八十度,满腹牢骚,法国,圆..."
3,3,3,人民日報_19580101_5_184206,1958,19580101,195801,第5版,NaN,扎记,NaN,NaN,又是他！,最近，美国新闻处发出了一条无地点、无日期的电讯。它引用了贾马利的一些言论，来污蔑正在开罗举行...,最近 ， 美国 新闻处 发出 了 一条 无 地点 、 无 日期 的 电讯 。 它 引用 了 ...,f x ns nt v ul m v n x v t uj n x r v ul nr uj...,"搜罗,贾马利,亚非,捣乱,团结,造谣生事,美国,亚非会议,贾马利来,悲哉,万隆会议,新闻处,..."
4,4,4,人民日報_19580101_5_184205,1958,19580101,195801,第5版,NaN,扎记,NaN,NaN,帮闲的嘴脸,当一年前埃及把西方殖民者的银行、公司收回来的时候，“纽约时报”曾经说：“由于丧失了外国资本、...,当 一年 前 埃及 把 西方 殖民者 的 银行 、 公司 收回 来 的 时候 ， “ 纽约时...,t m f ns p s n uj n x n v v uj n x x nt x d v ...,"殖民者,印度尼西亚,纽约时报,埃及,活不下去,援助,荷兰,管理,外国,米价,帮闲,挨饿,摇摇..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113141,17332,17332,人民日報_19621231_1_301127,1962,19621231,196212,第1版,NaN,NaN,新华社,NaN,我边防部队忠实执行我国政府停火后撤的决定国防部发言人声明：我边防部队于十二月三十日继续主动后...,新华社30日讯我国防部发言人今日就我边防部队继续从中印边界东西两段主动后撤发表声明如下：我西...,新华社 30 日 讯 我 国防部 发言人 今日 就 我 边防部队 继续 从中 印 边界 东西...,nt m m ng r nt l t d r n v d v n ns m b v l t ...,"边防部队,12,后撤,30,控制线,地区,1959,边界,中印,以北,习惯线,山口,进驻,我..."
113142,17333,17333,人民日報_19621231_1_301126,1962,19621231,196212,第1版,NaN,NaN,新华社,NaN,西丽玛沃·班达拉奈克总理简历,新华社30日讯锡兰总理西丽玛沃·班达拉奈克阁下1916年4月17日生于锡兰康堤的一个贵族家庭...,新华社 30 日讯 锡兰 总理 西丽玛沃 · 班达拉奈克 阁下 1916 年 4 月 17 ...,nt m nz ns n nr x ns n m m m m m m v ns nr uj ...,"班达拉奈克,锡兰,西丽玛沃,总理,丈夫,夫人,1960,科伦坡,国防,议会,妇女,协会,30..."
113143,17334,17334,人民日報_19621231_1_301125,1962,19621231,196212,第1版,NaN,NaN,新华社广州30日,NaN,锡兰总理班达拉奈克夫人到广州数千广州市民到车站热烈欢迎广东省省长设宴欢迎贵宾,新华社广州30日电锡兰总理西丽玛沃·班达拉奈克夫人和随行人员今天下午从深圳乘专车到达广州。贵...,新华社 广州 30 日电 锡兰 总理 西丽玛沃 · 班达拉奈克 夫人 和 随行人员 今天下午...,nt ns m j ns n nr x ns n c n nr p ns v n v ns ...,"锡兰,班达拉奈克,贵宾,陈郁,夫人,总理,西丽玛沃,广州,广东省,车站,曾生,省长,广州市,..."
113144,17335,17335,人民日報_19621231_4_301139,1962,19621231,196212,第4版,NaN,NaN,NaN,NaN,大会通电,首都各界人民庆祝古巴革命胜利四周年和中古友协成立，全文如下：首都各界人民今天举行了隆重集会，...,首都 各界 人民 庆祝 古巴 革命胜利 四周 年 和 中古 友协 成立 ， 全文如下 ： 首...,d r n v ns nz m m c ns j v x l x d r n t v ul ...,"古巴,美帝国主义,革命,菲德尔,卡斯特罗,英雄,革命胜利,拉丁美洲,各界,庆祝,坚强,胜利,..."


In [ ]:
# 先把縣市新聞先歸納出來
area = pd.read_excel('area.xlsx')
new_df = []


for k in list(area.columns):
  a = list(area[k])
  temp_ = []
  for j in range(len(a)):
    if type(a[j]) == str:
      if len(a[j])>=3:
        # b = a[j].replace('自治区','').replace('自治州','').replace('自治市','').replace('自治市','').replace('自治县','').replace('自治旗','').replace('区','').replace('市','').replace('县','').replace('旗','') 
        b = a[j].replace('自治区','').replace('自治州','').replace('自治市','').replace('自治市','').replace('自治县','').replace('自治旗','')
        temp_.append(b)
      else:
        temp_.append(a[j])
    else:
      continue
  new_df.append(temp_)


# 一次定義多變數

In [ ]:
for i,j in zip(list(area.columns), new_df):
    globals()[str(i)] = j

whole_region = 北京+天津+河北+山西+内蒙+辽宁+吉林+黑龙江+上海+江苏+浙江+安徽+福建+江西+山东+河南+湖北+广东+广西+四川+贵州+云南+陕西+甘肃+青海+新疆+湖南 # 全部行政區    
print(天津)


['和平区', '河东区', '河西区', '南开区', '河北区', '红桥区', '东丽区', '西青区', '津南区', '北辰区', '武清区', '宝坻区', '滨海新区', '宁河区', '静海区', '蓟州区']


# 特殊地名微調

In [ ]:
天津 = ['和平区' if x=='和平' else x for x in 天津]
辽宁 = ['和平区' if x=='和平' else x for x in 辽宁]
广东 = ['和平县' if x=='和平' else x for x in 广东]
河北 = ['新华区' if x=='新华' else x for x in 河北]
河南 = ['新华区' if x=='新华' else x for x in 河南]

# 文章篩選

In [ ]:

北京_list, 天津_list, 河北_list, 山西_list, 内蒙_list, 辽宁_list, 吉林_list, 黑龙江_list, 上海_list, 江苏_list, 浙江_list, 安徽_list, 福建_list, 江西_list, 山东_list, 河南_list, 湖北_list, 广东_list, 广西_list, 四川_list, 贵州_list, 云南_list, 陕西_list, 甘肃_list, 青海_list, 新疆_list, 湖南_list = [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []
region_list = [北京, 天津, 河北, 山西, 内蒙, 辽宁, 吉林, 黑龙江, 上海, 江苏, 浙江, 安徽, 福建, 江西, 山东, 河南, 湖北, 广东, 广西, 四川, 贵州, 云南, 陕西, 甘肃, 青海, 新疆, 湖南]
region_dict_list = [北京_list, 天津_list, 河北_list, 山西_list, 内蒙_list, 辽宁_list, 吉林_list, 黑龙江_list, 上海_list, 江苏_list, 浙江_list, 安徽_list, 福建_list, 江西_list, 山东_list, 河南_list, 湖北_list, 广东_list, 广西_list, 四川_list, 贵州_list, 云南_list, 陕西_list, 甘肃_list, 青海_list, 新疆_list, 湖南_list]
for i in df['doc_content']:
  for k,z in zip(region_list, region_dict_list):
    for j in k: # 字典
      if j in i: 
        z.append(i)
        break
      else:
        continue


In [ ]:
region_dict_list = [北京_list, 天津_list, 河北_list, 山西_list, 内蒙_list, 辽宁_list, 吉林_list, 黑龙江_list, 上海_list, 江苏_list, 浙江_list, 安徽_list, 福建_list, 江西_list, 山东_list, 河南_list, 湖北_list, 广东_list, 广西_list, 四川_list, 贵州_list, 云南_list, 陕西_list, 甘肃_list, 青海_list, 新疆_list, 湖南_list]

In [ ]:
num = 0
for i in region_dict_list:
  print(str(num) + '：' + str(len(i)))
  num += 1

0：460
1：255
2：1647
3：5708
4：877
5：1655
6：899
7：3417
8：181
9：1462
10：740
11：4092
12：1090
13：869
14：1341
15：2355
16：1608
17：1746
18：767
19：2510
20：785
21：1024
22：1025
23：2416
24：460
25：540
26：1890


In [ ]:
天津

['和平区',
 '河东区',
 '河西区',
 '南开区',
 '河北区',
 '红桥区',
 '东丽区',
 '西青区',
 '津南区',
 '北辰区',
 '武清区',
 '宝坻区',
 '滨海新区',
 '宁河区',
 '静海区',
 '蓟州区']

In [ ]:
河北_list[2]

'新华社唐山3日电中共唐山市委在最近召开的有各厂矿党委领导干部参加的市委扩大会议上，检查了整改和工业生产高潮准备工作。会议上，批判了某些厂矿企业领导干部对即将到来的工业生产高潮估计不足，干劲不大的右倾保守思想，号召各厂矿的干部和职工鼓起革命干劲，保证今年第一季度的生产不低于去年四季度的实际生产水平。唐山市随着厂矿企业整改高潮而来的，一个新的规模壮阔的工业生产高潮正在形成。开滦林西矿有十二个掘进队开展同工种生产竞赛后，到去年12月26日就有九个队的工人实现了全月竞赛条件中规定的保证计划。启新水泥厂工人，从去年12月初就开展起磨与磨、班与班的竞赛，生产水平不断提高，到28日提前完成了去年第四季度增产五千吨水泥计划。会议指出，目前有些厂矿企业的领导干部对新形势估计不足，他们认为即将到来的工业生产高潮和往年的增产节约运动不会有多大区别；他们没有认识到经过这次整风运动和社会主义教育运动，工人的政治觉悟和劳动热情高于过去的任何一次政治运动。因此他们在新的工业生产高潮前面表现了领导气魄不大，干劲不足，强调困难多，初步拟定的1958年的一些生产指标偏低。开滦煤矿唐家庄矿根据去年的实际生产水平稍加努力就能使1958年的产量提高到二百零七万吨，但是开滦煤矿只分配给这个矿一百九十万吨的生产任务，这个数字比去年的实际生产数字还少二万吨。唐山钢厂去年每炼一吨钢的平均实际生铁消耗已降到一千一百八十公斤，但这个厂在制定今年第一季度的生产计划时，强调供应的生铁可能含磷高，消耗大，因而规定第一季度每一吨转炉钢的生铁消耗为一千一百九十公斤，比去年的实际消耗还多十公斤。这次会议要求各厂矿领导干部要以革命的干劲，领导好新的工业生产高潮，并且要充分做好生产高潮的准备工作。在今年1月份，要发动职工群众，重新修订1958年的特别是第一季度的增产节约计划。会议要求凡是生产原材料的厂矿，在保证质量的前提下尽量增产；原材料供应有困难的厂矿，在保证产品质量的原则下想尽一切办法降低原材料消耗，研究试用代用品，节约原料保证增产；凡是产供销都有困难的厂矿，要积极承揽订货；并试制适应市场需要的新产品。同时各厂矿要坚决贯彻一竿子扎到底的整改方针，彻底作好系统整改工作，及时发动群众掀起生产高潮。'

In [ ]:
# https://bbs.creaders.net/politics/bbsviewer.php?trd_id=831648&language=big5

# http://ww2.usc.cuhk.edu.hk/PaperCollection/Details.aspx?id=362

In [ ]:
# 科爾估算的死亡率 與整體災害報導、高產衛星報導做比較

In [ ]:
# 蔣政華估算的死亡率 與整體災害報導、高產衛星報導做比較

In [ ]:
# 官方估計的死亡率 與整體災害報導、高產衛星報導做比較

In [ ]:
# 曹樹基的各省死亡率。 與各省的災害報導、高產衛星報導做比較

In [ ]:
# 楊繼繩，歷年鋼產量vs高產衛星。

In [ ]:
# 楊繼繩，歷年農產量vs高產衛星。